# EDA

- Joseph 2022-09-01

In [ ]:
import os
import subprocess
from pathlib import Path
from datetime import datetime

from tqdm import tqdm
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Count the number of files
csv_files = os.listdir("data")
print(f"There are {len(csv_files)} csv files.")

In [ ]:
# Use linux command `awk` to fast analyze the data

data_timerange = {
    "pair_name": [],
    "base": [],
    "quote": [],
    "start_time": [],
    "end_time": []
}
for csv_file in tqdm(csv_files):
    pair_name = csv_file.replace(".csv", "")
    if "-" in pair_name:
        base_coin, quote_coin = pair_name.split("-")
    else:
        base_coin = pair_name[:-3]
        quote_coin = pair_name[-3:]

    cmd = "awk -F ',' 'NR==2 {print $1} END {print $1}' data/" + csv_file
    p = subprocess.Popen(cmd, stdin=subprocess.PIPE, stdout=subprocess.PIPE, shell=True)
    dt = p.stdout.read()[:-1].decode("utf-8")
    if dt.count("\n") != 1:
        continue
    start_str, end_str = dt.split("\n")
    start_dt = datetime.fromtimestamp(int(start_str)/1000)
    end_dt = datetime.fromtimestamp(int(end_str)/1000)

    data_timerange["pair_name"].append(pair_name)
    data_timerange["base"].append(base_coin)
    data_timerange["quote"].append(quote_coin)
    data_timerange["start_time"].append(start_dt)
    data_timerange["end_time"].append(end_dt)
data_timerange = pd.DataFrame(data_timerange)

# Use pandas to analyze the data (less efficient)

data_timerange = {
    "pair_name": [],
    "base": [],
    "quote": [],
    "start_time": [],
    "end_time": []
}
for csv_file in csv_files:
    pair_name = csv_file.replace(".csv", "")
    if "-" in pair_name:
        base_coin, quote_coin = pair_name.split("-")
    else:
        base_coin = pair_name[:-3]
        quote_coin = pair_name[-3:]
    df = pd.read_csv(f"data/{csv_file}")
    if df.empty:
        continue
    start_dt = datetime.fromtimestamp(int(df.iloc[0]["time"]/1000))
    end_dt = datetime.fromtimestamp(int(df.iloc[-1]["time"]/1000))
    print(f"{pair_name}: {start_dt} - {end_dt}")
    data_timerange["pair_name"].append(pair_name)
    data_timerange["base"].append(base_coin)
    data_timerange["quote"].append(quote_coin)
    data_timerange["start_time"].append(start_dt)
    data_timerange["end_time"].append(end_dt)
data_timerange = pd.DataFrame(data_timerange)

In [ ]:
data_timerange.head()

In [ ]:
data_timerange.to_csv("EDA_results/data_timerange.csv")

To develop a meaningful trading strategy, sufficient
historical data must be available.
Since many of the cryptocurrencies either emerge late
or fail to survive, they are not qualified as the
trading instruments. Here we are only interested in
those that have a complete history from 2020-01-01 to 2022-08-10.

We are going to use 2 years' historical data (
2020-01-01 - 2021-12-31) as the training dataset to develop our
strategy; while leaving the 8 months' data (2022-01-01 -
2022-08-10) as the testing dataset. We will develop a
strategy denominated on `USD`, therefore we
only consider `USD` as the quote currency.

We can find out the number of cryptocurrency that satisfies
these criteria is 73:

In [21]:
data_timerange_selected = data_timerange[
    (data_timerange.start_time <= "2020-01-01")
    & (data_timerange.end_time >= "2022-08-10")
    & (data_timerange.quote == "usd")
].reset_index(inplace=False)
print(f"Number of pairs that are qualified: {data_timerange_selected.shape[0]}")

Number of pairs that are qualified: 73


Prepare data files for backtest:

In [ ]:
Path("clean_data/k_line/K_1M").mkdir(parents=True, exist_ok=True)

In [ ]:
for i, row in tqdm(data_universe.iterrows()):
    df = pd.read_csv(f"data/{row.pair_name}.csv")
    df["time_key"] = df.time.apply(lambda x: datetime.fromtimestamp(int(x/1000)))
    df["date"] = df.time_key.apply(lambda x: x.date())
    df = df[
        (df.time_key >= "2020-01-01")
        & (df.time_key <= "2022-08-10")
    ]
    # print(f"{i}. {row.pair_name}: {df.shape}")
    for date_ in df.date.unique():
        df_daily = df[df.date == date_][
            ["time_key", "open", "high", "low", "close", "volume"]
        ].reset_index(drop=True)
        date_str = date_.strftime("%Y-%m-%d")
        Path(f"clean_data/k_line/K_1M/{row.pair_name}").mkdir(parents=True, exist_ok=True)
        df_daily.to_csv(f"clean_data/k_line/K_1M/{row.pair_name}/{date_str}.csv",
                        index=False)

We want to see the active trading hours of each coin.

In [ ]:
num_bars_in_1hr = {}
for i, row in tqdm(data_universe.iterrows()):
    # if i > 7:
    #     break
    if i % 4 == 0:
        fig, axes = plt.subplots(
            nrows=4, #data_universe.shape[0],
            ncols=1,
            sharex=True
        )
    df = pd.read_csv(f"data/{row.pair_name}.csv")
    df["time_key"] = df.time.apply(lambda x: datetime.fromtimestamp(int(x/1000)))
    df["date"] = df.time_key.apply(lambda x: x.date())
    df["hour"] = df.time_key.apply(lambda x: x.hour)
    df = df[
        (df.time_key >= "2020-01-01")
        & (df.time_key <= "2022-08-10")
    ]
    num_days = len(df.date.unique())
    # Average number of bars in that particular hour
    df1 = df.groupby("hour").count().time_key / num_days
    df1.plot(ax=axes[i%4], title=f"{row.pair_name}")
    num_bars_in_1hr[row.pair_name] = {
        "max": df1.max(),
        "min": df1.min(),
        "mean": df1.mean(),
        "median": df1.median(),
        "std": df1.std()
    }
num_bars_in_1hr = pd.DataFrame(num_bars_in_1hr).T
num_bars_in_1hr = num_bars_in_1hr.sort_values(by="min", ascending=False)
num_bars_in_1hr

We can see that `BTC` and `ETH` are trading most actively; while
coins such as `LYM` and `PAS` are almost inactive. The table shows
that there are 59 bars (1-min OHLCV) in every hour for `BTC`, and 57
bars for `ETH`, which indicates high liquidity for these two
cryptocurrencies. The less this number becomes, the more illiquid
it means. Let's list the top 10 coins:


In [ ]:
num_bars_in_1hr.head(10)

As we can see, only the first 6 pairs are sufficient liquid (with
more than 45 bars in an hour). There is a big gap between `LTC`
and `EOS`. In order to guarantee the liquidity of trading
instruments, we limit our trading universe to the top 6 cryptocurrency
pairs, i.e., `BTCUSD`, `ETHUSD`, `XRPUSD`, `LEOUSD`, `USTUSD`,
and `LTCUSD`.

We can take a closer look at these coins:

In [ ]:
pairs = {
    "btcusd": None,
    "ethusd": None,
    "xrpusd": None,
    "leousd": None,
    "ustusd": None,
    "ltcusd": None
}
for i, pair_name in enumerate(pairs):
    print(i, i%2, pair_name)
    if i % 2 == 0:
        fig, axes = plt.subplots(
            nrows=2, #data_universe.shape[0],
            ncols=1,
            sharex=True
        )
    df = pd.read_csv(f"data/{pair_name}.csv")
    df["time_key"] = df.time.apply(lambda x: datetime.fromtimestamp(int(x/1000)))
    df["date"] = df.time_key.apply(lambda x: x.date())
    df["hour"] = df.time_key.apply(lambda x: x.hour)
    df = df[
        (df.time_key >= "2020-01-01")
        & (df.time_key <= "2022-08-10")
    ]
    pairs[pair_name] = df.copy()
    num_days = len(df.date.unique())
    # Average number of bars in that particular hour
    df1 = df.groupby("hour").count().time_key / num_days
    df1.plot(ax=axes[i%2], title=f"{pair_name}")

As we can see, all six pairs have similar active trading
hours. The market is relatively silent from 9 am to 4 pm.
This may indicate that the investors of different cryptocurrencies
are likely the same group of people.

Let's look at the price movements of these cryptocurrencies:

In [ ]:
for i, pair_name in enumerate(pairs):
    if i % 2 == 0:
        fig, axes = plt.subplots(
            nrows=2, #data_universe.shape[0],
            ncols=1,
            sharex=True
        )
    df = pairs[pair_name]
    df[["time_key", "close"]].set_index("time_key").plot(
        ax=axes[i%2],
        title=pair_name)

We can see that `BTC`, `ETH`, `XRP`, and `LTC` show similarity
in price movement, but `LEO` and `UST` differ from the others.
We can see it more clear in a correlation heatmap:

In [ ]:
all_pairs = None
for pair_name in pairs:
    df = pairs[pair_name][["time_key", "close"]].set_index("time_key")
    print(pair_name, df.shape)
    # if all_pairs is None:
    #     all_pairs = df.copy()
    # else:
    #     all_pairs = all_pairs.merge(df, how="outer")